In [79]:
using ControlSystems
using DistributedControlSystems
using RobustAndOptimalControl
using LinearAlgebra
using BlockArrays

In [260]:
# patch
function control_form(sys)
    A, B, C = sys.A, sys.B, sys.C
    n = size(A, 1)
    tsys = tf(sys)
    α = tsys.matrix[1].den.coeffs
    
    P = zeros(n, n)
    P[:, n] = B
    for i in n - 1:-1:1
        p = P[:, i + 1]
        P[:, i] = A * p + α[i + 1] * B
    end

    similarity_transform(sys, P), P
end

function relative_degree(sys)
    A, B, C = sys.A, sys.B, sys.C
    n = size(A, 1)
    for r in 1:n
        u_to_dy = C * A^(r - 1) * B
        # !(u_to_dy[1] ≈ 0)
        if (!isapprox(u_to_dy[1], 0, atol=1e-10))
            return r, u_to_dy[1]
        end
    end
    
end

function null_form(sys)
    sys, T = control_form(sys)
    A, B, C = sys.A, sys.B, sys.C
    n = size(A, 1)
    r, R = relative_degree(sys)
    P = diagm(0 => ones(n))
    
    C = @. C / R
    
    for i in 1:n - r
        @show i, n, r
        P[n - r + 1:end, :] += diagm(r, n, i - 1 => fill(C[i], r))
    end
    P = inv(P)
    sys = similarity_transform(sys, P)
    # sys = ss(sys.A, sys.B, sys.C * R, 0)
    sys, P, r
end

function null_split(sys)
    null_sys, _, r = null_form(sys)
    A, B, C = null_sys.A, null_sys.B, null_sys.C
    n = size(A, 1)
    
    T = diagm(r, n, n - r => fill(1, r))
    pT = pinv(T)
    N = diagm(n - r, n, 0 => fill(1, n - r))
    pN = pinv(N)
    
    yA = T * A * pT
    yB = T * B
    yC = C * pT
    y_sys = ss(yA, yB, yC, 0)
    
    nA = A[1:n - r, 1:n - r]
    nB = zeros(n - r, 1)
    nB[end] = 1 / yC[1]
    nC = A[n:n, 1:n - r]
    
    n_sys = ss(nA, nB, nC, 0)
    
    y_sys, n_sys
end

null_split (generic function with 1 method)

# Проверка масштабируемости на задаче консенсуса.

Оказалось, что уже при 10 агентах задача не решается.

Это связано с ростом контроллера на каждой итерации, так как предыдущие контроллеры связали первых агентов с текущим:
Они стали наблюдаемыми и управляемыми.

Возможное решение - выделение нулевой динамики на каждой итерации. Если она устойчивая, то можно создать контроллер с размерностью равной относительному порядку.

In [80]:
a = [
    0 1
    0 0
]
b = [
    0
    1
]
c = [1 0]

1×2 Matrix{Int64}:
 1  0

In [81]:
N = 10

10

In [82]:
A = kron(I(N), a)

20×20 Matrix{Int64}:
 0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  

In [83]:
Bs = eachcol(kron(I(N), b)) .|> Array

10-element Vector{Vector{Int64}}:
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [84]:
L = diagm(0 => 2ones(N), 1 => -ones(N - 1), -1 => -ones(N - 1))
L[1, 1] = 1
L[end, end] = 1
L

10×10 Matrix{Float64}:
  1.0  -1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
 -1.0   2.0  -1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
  0.0  -1.0   2.0  -1.0   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0  -1.0   2.0  -1.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0  -1.0   2.0  -1.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0  -1.0   2.0  -1.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0  -1.0   2.0  -1.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0  -1.0   2.0  -1.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0  -1.0   2.0  -1.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  -1.0   1.0

In [85]:
Cs = eachrow(kron(L, c)) .|> Array .|> permutedims

10-element Vector{Matrix{Float64}}:
 [1.0 0.0 … 0.0 0.0]
 [-1.0 -0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … -1.0 -0.0]
 [0.0 0.0 … 1.0 0.0]

In [86]:
ssd = StateSpaceDistributed(A, Bs, Cs)

StateSpaceDistributed{Matrix{Int64}, BlockMatrix{Int64, Matrix{Matrix{Int64}}, Tuple{BlockedUnitRange{Vector{Int64}}, BlockedUnitRange{Vector{Int64}}}}, BlockMatrix{Float64, Matrix{Matrix{Float64}}, Tuple{BlockedUnitRange{Vector{Int64}}, BlockedUnitRange{Vector{Int64}}}}, BlockMatrix{Int64, Matrix{Matrix{Int64}}, Tuple{BlockedUnitRange{Vector{Int64}}, BlockedUnitRange{Vector{Int64}}}}}([0 1 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 1; 0 0 … 0 0], [0 0 … 0 0; 1 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 1], [1.0 0.0 … 0.0 0.0; -1.0 -0.0 … 0.0 0.0; … ; 0.0 0.0 … -1.0 -0.0; 0.0 0.0 … 1.0 0.0], [0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0])

In [87]:
fixed_modes(ssd; digits=5)

2-element Vector{ComplexF64}:
 0.0 - 0.0im
 0.0 + 0.0im

In [93]:
cs, Acl = distributed_controllers(ssd)

LoadError: The Hamiltonian matrix is not dichotomic

In [94]:
c1, s1 = DistributedControlSystems.local_controller_and_closed_sys(ssd, 1, ssd.A)
A1 = s1.A
@show c1
A1

c1 = StateSpace{Continuous, Float64}
A = 
 -1.7320508075688783  -2.000000000000001
  1.0                 -1.7320508075688774
B = 
 -1.0000000000000007
 -1.7320508075688774
C = 
 -1.7320508075688783  -1.0000000000000004
D = 
 0.0

Continuous-time state-space model


22×22 Matrix{Float64}:
  0.0      1.0  0.0      0.0  0.0  0.0  …  0.0  0.0  0.0  -0.0      -0.0
  0.0      0.0  0.0      0.0  0.0  0.0     0.0  0.0  0.0   1.73205   1.0
  0.0      0.0  0.0      1.0  0.0  0.0     0.0  0.0  0.0  -0.0      -0.0
  0.0      0.0  0.0      0.0  0.0  0.0     0.0  0.0  0.0  -0.0      -0.0
  0.0      0.0  0.0      0.0  0.0  1.0     0.0  0.0  0.0  -0.0      -0.0
  0.0      0.0  0.0      0.0  0.0  0.0  …  0.0  0.0  0.0  -0.0      -0.0
  0.0      0.0  0.0      0.0  0.0  0.0     0.0  0.0  0.0  -0.0      -0.0
  0.0      0.0  0.0      0.0  0.0  0.0     0.0  0.0  0.0  -0.0      -0.0
  0.0      0.0  0.0      0.0  0.0  0.0     0.0  0.0  0.0  -0.0      -0.0
  0.0      0.0  0.0      0.0  0.0  0.0     0.0  0.0  0.0  -0.0      -0.0
  0.0      0.0  0.0      0.0  0.0  0.0  …  0.0  0.0  0.0  -0.0      -0.0
  0.0      0.0  0.0      0.0  0.0  0.0     0.0  0.0  0.0  -0.0      -0.0
  0.0      0.0  0.0      0.0  0.0  0.0     0.0  0.0  0.0  -0.0      -0.0
  0.0      0.0  0.0      0.0

In [95]:
ctrb(s1)

22×22 Matrix{Float64}:
 0.0  1.0   0.0       0.0       -3.4641   …   89.0     -93.5307   72.0
 1.0  0.0   0.0      -3.4641    11.0         -93.5307   72.0     -27.7128
 0.0  0.0   0.0       0.0        0.0           0.0       0.0       0.0
 0.0  0.0   0.0       0.0        0.0           0.0       0.0       0.0
 0.0  0.0   0.0       0.0        0.0           0.0       0.0       0.0
 0.0  0.0   0.0       0.0        0.0      …    0.0       0.0       0.0
 0.0  0.0   0.0       0.0        0.0           0.0       0.0       0.0
 0.0  0.0   0.0       0.0        0.0           0.0       0.0       0.0
 0.0  0.0   0.0       0.0        0.0           0.0       0.0       0.0
 0.0  0.0   0.0       0.0        0.0           0.0       0.0       0.0
 0.0  0.0   0.0       0.0        0.0      …    0.0       0.0       0.0
 0.0  0.0   0.0       0.0        0.0           0.0       0.0       0.0
 0.0  0.0   0.0       0.0        0.0           0.0       0.0       0.0
 0.0  0.0   0.0       0.0        0.0           0.0 

In [96]:
A1

22×22 Matrix{Float64}:
  0.0      1.0  0.0      0.0  0.0  0.0  …  0.0  0.0  0.0  -0.0      -0.0
  0.0      0.0  0.0      0.0  0.0  0.0     0.0  0.0  0.0   1.73205   1.0
  0.0      0.0  0.0      1.0  0.0  0.0     0.0  0.0  0.0  -0.0      -0.0
  0.0      0.0  0.0      0.0  0.0  0.0     0.0  0.0  0.0  -0.0      -0.0
  0.0      0.0  0.0      0.0  0.0  1.0     0.0  0.0  0.0  -0.0      -0.0
  0.0      0.0  0.0      0.0  0.0  0.0  …  0.0  0.0  0.0  -0.0      -0.0
  0.0      0.0  0.0      0.0  0.0  0.0     0.0  0.0  0.0  -0.0      -0.0
  0.0      0.0  0.0      0.0  0.0  0.0     0.0  0.0  0.0  -0.0      -0.0
  0.0      0.0  0.0      0.0  0.0  0.0     0.0  0.0  0.0  -0.0      -0.0
  0.0      0.0  0.0      0.0  0.0  0.0     0.0  0.0  0.0  -0.0      -0.0
  0.0      0.0  0.0      0.0  0.0  0.0  …  0.0  0.0  0.0  -0.0      -0.0
  0.0      0.0  0.0      0.0  0.0  0.0     0.0  0.0  0.0  -0.0      -0.0
  0.0      0.0  0.0      0.0  0.0  0.0     0.0  0.0  0.0  -0.0      -0.0
  0.0      0.0  0.0      0.0

In [98]:
fixed_modes(A1, [ssd.B[Block(2)]; zeros(2, 1)], [ssd.C[Block(2)] zeros(1, 2)])

16-element Vector{ComplexF64}:
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im

In [99]:
c2, s2 = DistributedControlSystems.local_controller_and_closed_sys(ssd, 2, A1)
A2 = s2.A
@show c2
A2

c2 = StateSpace{Continuous, Float64}
A = 
 -2.236278925189659   4.000471715623707   -0.15295134181409836  -0.031386600007709604   1.2042261936045109   -2.0230451678677297
 -1.0                -4.09986580571708     0.0                   0.0                   -2.0081018486359246    0.4120095529560263
  0.0                -1.4847441110333512  -2.1650635094610973    0.8884278944919166    -1.530086241032466     1.211566073781041
  0.0                 0.8946848229851884  -2.4622144504490273   -1.1561885545369572     0.20821865615082907  -1.210888573447794
  0.0                 1.8841793944729694   0.0                  -0.7181061759698143     1.0570297036970606    0.4645587289779854
  0.0                -0.5515468562930422   0.0                   0.0                   -0.32698192671333753  -0.22158706456421445
B = 
 -1.0000000000000009
  2.04993290285854
 -0.2576279444833246
 -0.4473424114925942
 -0.9420896972364847
  0.2757734281465211
C = 
 -2.236278925189659  2.000471715623705  -0.15295134

28×28 Matrix{Float64}:
  0.0       1.0   0.0       0.0  …  -0.0        -0.0       -0.0
  0.0       0.0   0.0       0.0     -0.0        -0.0       -0.0
  0.0       0.0   0.0       1.0     -0.0        -0.0       -0.0
  0.0       0.0   0.0       0.0      0.0313866  -0.224632   1.82206
  0.0       0.0   0.0       0.0     -0.0        -0.0       -0.0
  0.0       0.0   0.0       0.0  …  -0.0        -0.0       -0.0
  0.0       0.0   0.0       0.0     -0.0        -0.0       -0.0
  0.0       0.0   0.0       0.0     -0.0        -0.0       -0.0
  0.0       0.0   0.0       0.0     -0.0        -0.0       -0.0
  0.0       0.0   0.0       0.0     -0.0        -0.0       -0.0
  0.0       0.0   0.0       0.0  …  -0.0        -0.0       -0.0
  0.0       0.0   0.0       0.0     -0.0        -0.0       -0.0
  0.0       0.0   0.0       0.0     -0.0        -0.0       -0.0
  ⋮                              ⋱   ⋮                     
  0.0       0.0   0.0       0.0     -0.0        -0.0       -0.0
  0.0       0.0  

In [77]:
sum(ctrb(s1), dims=2)

22×1 Matrix{Float64}:
  28.00515477614499
   0.2923418550385861
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
  -0.5602910474696472
 -28.737205583713887

In [78]:
sum(ctrb(s2), dims=2)

28×1 Matrix{Float64}:
 -2.5396156955544563e10
  6.329426318216775e10
  2.7991813249534378e10
 -6.98317259172932e10
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
 -8.732823088799321e10
 -6.600634182029927e9
  8.434800239661188e10
  4.056664482593897e9
  3.774424091685689e10
  8.501634094522588e10
  3.711458945288602e10
 -3.551042621425168e9

In [103]:
k = 2
Ak = A1
B = ssd.B[Block(k)]
m = size(Ak, 1) - size(B, 1)
Bk = [
    B
    zeros(m, size(B, 2))
]
C = ssd.C[Block(k)]
n = size(Ak, 2) - size(C, 2)
Ck = [C zeros(size(C, 1), n)]

sys = ss(Ak, Bk, Ck, 0)
local_controller = DistributedControlSystems.robust_controller(sys)

StateSpace{Continuous, Float64}
A = 
 -2.236278925189659   4.000471715623707   -0.15295134181409836  -0.031386600007709604   1.2042261936045109   -2.0230451678677297
 -1.0                -4.09986580571708     0.0                   0.0                   -2.0081018486359246    0.4120095529560263
  0.0                -1.4847441110333512  -2.1650635094610973    0.8884278944919166    -1.530086241032466     1.211566073781041
  0.0                 0.8946848229851884  -2.4622144504490273   -1.1561885545369572     0.20821865615082907  -1.210888573447794
  0.0                 1.8841793944729694   0.0                  -0.7181061759698143     1.0570297036970606    0.4645587289779854
  0.0                -0.5515468562930422   0.0                   0.0                   -0.32698192671333753  -0.22158706456421445
B = 
 -1.0000000000000009
  2.04993290285854
 -0.2576279444833246
 -0.4473424114925942
 -0.9420896972364847
  0.2757734281465211
C = 
 -2.236278925189659  2.000471715623705  -0.1529513418140

In [106]:
reduced_system = sys |> minreal2

StateSpace{Continuous, Float64}
A = 
  0.0   0.0                  0.0                  0.0                  0.0                    0.0
 -1.0   0.0                  0.0                  0.0                  0.0                    0.0
  0.0  -2.0000000000000004  -2.1650635094610973   0.8884278944919166  -1.7824570141713456     1.2633459023244726
  0.0   0.0                 -2.4622144504490273  -1.1561885545369572  -0.22999525916263264   -1.1209786321417528
  0.0   0.0                  0.0                 -0.7181061759698143   0.1341643455366075     0.6539063678502746
  0.0   0.0                  0.0                  0.0                 -0.056835947743258876  -0.27701389667630905
B = 
 -1.0
  0.0
  0.0
  0.0
  0.0
  0.0
C = 
 0.0  2.0  0.0  0.0  0.9795939397995497  -0.20098684809707545
D = 
 0.0

Continuous-time state-space model

In [109]:
reduced_system.A |> eigvals

6-element Vector{ComplexF64}:
 -0.8660254420519817 - 0.4999999859012462im
 -0.8660254420519817 + 0.4999999859012462im
 -0.8660253655168982 - 0.5000000140987594im
 -0.8660253655168982 + 0.5000000140987594im
                 0.0 + 0.0im
                 0.0 + 0.0im

In [114]:
extended_system = ExtendedStateSpace(reduced_system)

ExtendedStateSpace{Continuous, Float64}
A = 
  0.0   0.0                  0.0                  0.0                  0.0                    0.0
 -1.0   0.0                  0.0                  0.0                  0.0                    0.0
  0.0  -2.0000000000000004  -2.1650635094610973   0.8884278944919166  -1.7824570141713456     1.2633459023244726
  0.0   0.0                 -2.4622144504490273  -1.1561885545369572  -0.22999525916263264   -1.1209786321417528
  0.0   0.0                  0.0                 -0.7181061759698143   0.1341643455366075     0.6539063678502746
  0.0   0.0                  0.0                  0.0                 -0.056835947743258876  -0.27701389667630905
B1 = 
 -1.0
  0.0
  0.0
  0.0
  0.0
  0.0
B2 = 
 -1.0
  0.0
  0.0
  0.0
  0.0
  0.0
C1 = 
 0.0  2.0  0.0  0.0  0.9795939397995497  -0.20098684809707545
C2 = 
 0.0  2.0  0.0  0.0  0.9795939397995497  -0.20098684809707545
D11 = 
 0.0
D12 = 
 0.0
D21 = 
 0.0
D22 = 
 0.0
Continuous-time extended state-space m

In [118]:
lqg = LQGProblem(extended_system, I(1), I(1), I(1), I(1))

LQGProblem(ExtendedStateSpace{Continuous, Float64}
A = 
  0.0   0.0                  0.0                  0.0                  0.0                    0.0
 -1.0   0.0                  0.0                  0.0                  0.0                    0.0
  0.0  -2.0000000000000004  -2.1650635094610973   0.8884278944919166  -1.7824570141713456     1.2633459023244726
  0.0   0.0                 -2.4622144504490273  -1.1561885545369572  -0.22999525916263264   -1.1209786321417528
  0.0   0.0                  0.0                 -0.7181061759698143   0.1341643455366075     0.6539063678502746
  0.0   0.0                  0.0                  0.0                 -0.056835947743258876  -0.27701389667630905
B1 = 
 -1.0
  0.0
  0.0
  0.0
  0.0
  0.0
B2 = 
 -1.0
  0.0
  0.0
  0.0
  0.0
  0.0
C1 = 
 0.0  2.0  0.0  0.0  0.9795939397995497  -0.20098684809707545
C2 = 
 0.0  2.0  0.0  0.0  0.9795939397995497  -0.20098684809707545
D11 = 
 0.0
D12 = 
 0.0
D21 = 
 0.0
D22 = 
 0.0
Continuous-time extended st

In [119]:
clqg = observer_controller(lqg)

StateSpace{Continuous, Float64}
A = 
 -1.8433665258726637   3.699000074353929      0.24641266014171792  -0.2881057825945479   1.8545655428786625     0.5857154014088015
 -1.0                 -1.9359511088686796     0.0                   0.0                 -0.9482229869979883     0.19455035572077706
  0.0                 -1.1121473713743197    -2.1650635094610973    0.8884278944919166  -1.3475894869529372     1.1741225516233833
  0.0                 -0.8159183561170631    -2.4622144504490273   -1.1561885545369572  -0.6296295976743755    -1.0389842027914948
  0.0                  0.18931988984258943    0.0                  -0.7181061759698143   0.22689265392326696    0.6348809638795008
  0.0                  0.001430196970694071   0.0                   0.0                 -0.056135441600653084  -0.27715762206695793
B = 
 -1.0000000000000007
  0.9679755544343398
 -0.44392631431284035
  0.40795917805853155
 -0.09465994492129472
 -0.0007150984853470355
C = 
 -1.8433665258726637  1.699000074

In [124]:
feedback(reduced_system * clqg)

StateSpace{Continuous, Float64}
A = 
  0.0   0.0                    0.0                  0.0                  0.0                     0.0                      1.8433665258726637  -1.6990000743539277    -0.24641266014171792   0.2881057825945479  -0.8749716030791121    -0.7867022495058771
 -1.0   0.0                    0.0                  0.0                  0.0                     0.0                      0.0                  0.0                    0.0                   0.0                  0.0                    0.0
  0.0  -2.0000000000000004    -2.1650635094610973   0.8884278944919166  -1.7824570141713456      1.2633459023244726       0.0                  0.0                    0.0                   0.0                  0.0                    0.0
  0.0   0.0                   -2.4622144504490273  -1.1561885545369572  -0.22999525916263264    -1.1209786321417528       0.0                  0.0                    0.0                   0.0                  0.0                    0.0
  0.

In [167]:
# Сделать выделение нулевой динамики
# Добавить супер-твистинг?

In [237]:
reduced_system |> control_form

(StateSpace{Continuous, Float64}
A = 
  2.2575304447792737e-15   1.000000000000009       -6.863004913946304e-15   -3.566704091760069e-15   -3.082608053611384e-15   -2.495449921460907e-15
 -1.941118264790306e-15   -2.8186038728740023e-15   1.0000000000000018       6.461343385004182e-16    9.514706468837945e-16    6.957575307680287e-16
  3.0129177767207246e-15   1.5304127922090057e-16  -1.6634733491587974e-17   0.9999999999999999       1.0069335839344905e-16  -1.8476141170789822e-17
 -4.411873903920122e-15    2.674273310084397e-16    9.138839663987617e-16    5.630825549967621e-16    1.0000000000000002       5.1284823880668084e-17
  4.527765063402887e-15   -2.0984955481576198e-15  -4.000263044241811e-15   -2.2204460492502914e-16  -1.3322676295501878e-15   1.0
 -2.654105094820774e-15    6.687785577534812e-15   -0.9999999999999928      -3.464101615137755       -5.000000000000001       -3.464101615137757
B = 
  2.7619295370235415e-16
 -7.650928633962859e-17
  1.0929317211635208e-17
 -3.80856

In [238]:
relative_degree(control_form(reduced_system)[1])

(2, 2.0)

In [261]:
s, P, r = reduced_system |> null_form
s

(i, n, r) = (1, 6, 2)
(i, n, r) = (2, 6, 2)
(i, n, r) = (3, 6, 2)
(i, n, r) = (4, 6, 2)


StateSpace{Continuous, Float64}
A = 
 -5.234115801290986e-16    1.0000000000000007      -3.0350179111315914e-14  -1.0356386004634585e-14   5.56188404981673e-15    -2.495449921460907e-15
 -1.2117661951933087e-15  -6.399329559629989e-16    1.0000000000000078       2.220446049250306e-15   -1.4587041391939909e-15   6.957575307680287e-16
  2.930569482288377e-15   -1.2298350994063954e-16  -8.081160625794418e-16    0.9999999999999994       1.6469658886469526e-16  -1.8476141170789822e-17
 -0.5000000000000058      -1.7320508075688787      -5.000000000000003       -3.4641016151377557       1.0                      5.1284823880668084e-17
  2.220446049250313e-15    0.0                     -1.7763568394002505e-15   8.881784197001252e-16    0.0                      1.0
 -5.55005388396397e-15    -2.88657986402541e-15    -0.5000000000000078      -1.7320508075688745       1.5383701491068518e-15  -4.440892098500626e-16
B = 
  2.7619295370235415e-16
 -7.650928633962859e-17
  1.0929317211635208e-17
 -3.80

In [262]:
tf(s)

TransferFunction{Continuous, ControlSystemsBase.SisoRational{Float64}}
                           2.0s^4 + 6.928203230275514s^3 + 10.000000000000016s^2 + 3.464101615137771s + 1.0000000000000135
-----------------------------------------------------------------------------------------------------------------------------------------------------
1.0s^6 + 3.4641016151377575s^5 + 5.000000000000007s^4 + 3.464101615137759s^3 + 1.0000000000000084s^2 + 1.600070281607879e-15s + 5.376997488551975e-16

Continuous-time transfer function model

In [263]:
tf(control_form(reduced_system)[1])

TransferFunction{Continuous, ControlSystemsBase.SisoRational{Float64}}
      1.3322676295501878e-15s^5 + 2.0000000000000036s^4 + 6.928203230275517s^3 + 10.000000000000009s^2 + 3.4641016151377677s + 1.0000000000000113
-------------------------------------------------------------------------------------------------------------------------------------------------------
1.0s^6 + 3.4641016151377575s^5 + 5.000000000000006s^4 + 3.4641016151377597s^3 + 1.0000000000000078s^2 + 1.1758087211998893e-15s + 5.329012734979561e-16

Continuous-time transfer function model

In [264]:
y_null, x_null = reduced_system |> null_split

(i, n, r) = (1, 6, 2)
(i, n, r) = (2, 6, 2)
(i, n, r) = (3, 6, 2)
(i, n, r) = (4, 6, 2)


(StateSpace{Continuous, Float64}
A = 
 0.0                      1.0
 1.5383701491068518e-15  -4.440892098500626e-16
B = 
 -5.5785056601052784e-18
  1.0
C = 
 2.0  0.0
D = 
 0.0

Continuous-time state-space model, StateSpace{Continuous, Float64}
A = 
 -5.234115801290986e-16    1.0000000000000007      -3.0350179111315914e-14  -1.0356386004634585e-14
 -1.2117661951933087e-15  -6.399329559629989e-16    1.0000000000000078       2.220446049250306e-15
  2.930569482288377e-15   -1.2298350994063954e-16  -8.081160625794418e-16    0.9999999999999994
 -0.5000000000000058      -1.7320508075688787      -5.000000000000003       -3.4641016151377557
B = 
 0.0
 0.0
 0.0
 0.5
C = 
 -5.55005388396397e-15  -2.88657986402541e-15  -0.5000000000000078  -1.7320508075688745
D = 
 0.0

Continuous-time state-space model)

In [265]:
s |> tf

TransferFunction{Continuous, ControlSystemsBase.SisoRational{Float64}}
                           2.0s^4 + 6.928203230275514s^3 + 10.000000000000016s^2 + 3.464101615137771s + 1.0000000000000135
-----------------------------------------------------------------------------------------------------------------------------------------------------
1.0s^6 + 3.4641016151377575s^5 + 5.000000000000007s^4 + 3.464101615137759s^3 + 1.0000000000000084s^2 + 1.600070281607879e-15s + 5.376997488551975e-16

Continuous-time transfer function model

In [266]:
feedback(y_null, -x_null) |> tf

TransferFunction{Continuous, ControlSystemsBase.SisoRational{Float64}}
      1.3322676295501878e-15s^5 + 2.0000000000000027s^4 + 6.928203230275518s^3 + 10.00000000000002s^2 + 3.4641016151377717s + 1.000000000000016
-----------------------------------------------------------------------------------------------------------------------------------------------------
1.0s^6 + 3.464101615137758s^5 + 5.000000000000006s^4 + 3.464101615137757s^3 + 1.0000000000000109s^2 + 3.718083693221438e-15s + 3.0739597530142573e-16

Continuous-time transfer function model

In [267]:
hinfnorm(s)

(Inf, 2.3118724428209306e-8)

In [268]:
hinfnorm2(y_null)

(Inf, NaN)

In [269]:
hinfnorm2(x_null)

(0.28668722212445413, 1.8673474555735576)

In [270]:
ispassive(x_null)

false

In [271]:
x_null

StateSpace{Continuous, Float64}
A = 
 -5.234115801290986e-16    1.0000000000000007      -3.0350179111315914e-14  -1.0356386004634585e-14
 -1.2117661951933087e-15  -6.399329559629989e-16    1.0000000000000078       2.220446049250306e-15
  2.930569482288377e-15   -1.2298350994063954e-16  -8.081160625794418e-16    0.9999999999999994
 -0.5000000000000058      -1.7320508075688787      -5.000000000000003       -3.4641016151377557
B = 
 0.0
 0.0
 0.0
 0.5
C = 
 -5.55005388396397e-15  -2.88657986402541e-15  -0.5000000000000078  -1.7320508075688745
D = 
 0.0

Continuous-time state-space model

In [278]:
y_lqg = LQGProblem(ExtendedStateSpace(y_null), I(1), I(1), I(1), I(1))
y_controller = observer_controller(y_lqg)

StateSpace{Continuous, Float64}
A = 
 -2.0000000000000044   1.0
 -4.000000000000002   -1.9999999999999991
B = 
 1.0000000000000022
 1.0000000000000013
C = 
 2.0000000000000004  1.9999999999999987
D = 
 0.0

Continuous-time state-space model

In [281]:
feedback(reduced_system, y_controller) |> poles

8-element Vector{ComplexF64}:
    -2.37980675582161 + 0.0im
  -1.7373975891414013 + 1.567548439260726im
  -1.7373975891414013 - 1.567548439260726im
  -0.4164243310403128 + 1.626035499596816im
  -0.4164243310403128 - 1.626035499596816im
 -0.11371370548206075 + 0.29369072335787816im
 -0.11371370548206075 - 0.29369072335787816im
  -0.5492236079885957 + 0.0im